# Гибриддная рекомендательная система 

Пакет моделирования - LightFM     
Набор данных - Goodreads

Данный блокнот возник в результате неудовлетворенности уроком по формирвоанию рекомендательной системы на основе пакета LighFM.


In [24]:
import pandas as pd, numpy as np



У меня на локальной машине установлена версия пакета LightFM без поддержки OpenMP, работать будет одно ядро, медленно и неспешно. 
Чтение данных - за основу берется оригинальный (!) набор данных, расположенный по адресу: https://github.com/zygmuntz/goodbooks-10k
Он содержит исходные xml файлы и скомпилированные csv.  Буду использовать скомпилированные csv& 

In [25]:
ratings = pd.read_csv('./goodbooks-10k-master/ratings.csv') #поставленные оценки
print('Ratings:',ratings.shape[0])
ratings.head()

Ratings: 5976479


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


Шесть (!) миллионов рекомендаций.


In [26]:

books = pd.read_csv('./goodbooks-10k-master/books.csv') #информация о книгах
print('Books: ', books.shape[0])
books.head()

Books:  10000


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [33]:

tags = pd.read_csv('./goodbooks-10k-master/tags.csv') #информация о тегах
print('Tags:', tags.shape[0], tags.tag_id.max())
tags.head()

Tags: 34252 34251


,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [28]:

book_tags = pd.read_csv('./goodbooks-10k-master/book_tags.csv') # информация о соотношении тэгов и книг
print('Book-tags combinations:', book_tags.shape[0])
book_tags.head()

Book-tags combinations: 999912


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


В оригинальном наборе есть еще одна интересная таблица - to_read.

In [29]:
to_read = pd.read_csv('./goodbooks-10k-master/to_read.csv')
print('To read:', to_read.shape[0])
to_read.head()

To read: 912705


,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


Фактически - это отобранные по итогам работы с сайтом GoodReads книги для чтения.  Оценок там пока что нет.  

Первая проблемка с данными - тэги размечены ссылкой на неправильный идентификатор книги. Надо добавить идентификатор из books. 


In [30]:
book_tags = book_tags.merge( books[[ 'goodreads_book_id', 'book_id']], on = 'goodreads_book_id' )
print(book_tags.shape[0])
book_tags['one'] = 1  #Служебное поле понадобится в двльнейшем
book_tags.head()

999912


,goodreads_book_id,tag_id,count,book_id,one
0,1,30574,167697,27,1
1,1,11305,37174,27,1
2,1,11557,34173,27,1
3,1,8717,12986,27,1
4,1,33114,12716,27,1


In [31]:
len(book_tags.tag_id.unique())

34252

Тэгов очень много!  Есть смысл посмотреть на них внимательнее.  Проанализирую частоту использования.

In [37]:
tags_qty=book_tags.groupby(['tag_id']).one.count()
tags['usage'] = tags.tag_id.map(tags_qty)
tags.head()

,tag_id,tag_name,usage
0,0,-,7
1,1,--1-,2
2,2,--10-,1
3,3,--12-,1
4,4,--122-,1


In [59]:
tags.sort_values(by='usage', ascending=True).head(20)

,tag_id,tag_name,usage
34251,34251,ｆａｖｏｕｒｉｔｅｓ,1
24730,24730,queer-fantasy,1
12360,12360,forensic-medicine-thriller,1
24724,24724,queen-elizabeth-ii,1
24723,24723,queen-elizabeth,1
12364,12364,forensic-sculptor,1
24719,24719,quebec-city,1
24717,24717,quarterbacks,1
24711,24711,quanah-parker,1
12368,12368,forest-fires,1


In [57]:
tags.sort_values(by='usage', ascending=False).head(20)

,tag_id,tag_name,usage
30574,30574,to-read,9983
11557,11557,favorites,9881
22743,22743,owned,9858
5207,5207,books-i-own,9799
8717,8717,currently-reading,9776
18045,18045,library,9415
22753,22753,owned-books,9221
11743,11743,fiction,9097
30521,30521,to-buy,8692
17213,17213,kindle,8316


Изначально, именно пользователи размечали книги тэгами. Возникло несколько проблем:
- Пользователи назначили очень много тэгов, и при этом многие из них используются 1-2 раза.  Проблема понимания смысла тэга и его переиспользования в принципе характерна для систем на основе неконтролируемого пространства тэгов. (А ведь пользователи забыли о таком очевидном тэге, как автор произведения, так как все книги размечены авторством.) 
- Некоторое количество тэгов размечает слишком много книг. Если почти все книги отмечены признаком 'fiction', то ценность этого признака в этом наборе снижается. 
Из всего набора тэгов есть смысл вырезать информативный кусок, границы которого и предстоит определить. 

In [58]:
for q in [0.25,0.5,0.75,0.8,0.95,0.99]:
    qq = tags.usage.quantile(q)
    s = tags[tags.usage > qq].shape[0]
    print(f'{q} : {qq}, {s}')


0.25 : 1.0, 17095
0.5 : 1.0, 17095
0.75 : 5.0, 7905
0.8 : 7.0, 6392
0.95 : 53.0, 1708
0.99 : 477.48999999999796, 343


In [62]:
low_usage = 10  
low_qty=tags[tags.usage<low_usage].shape[0]
print(f'Low usage limit:{low_usage}, {low_qty} tags to delete')
high_usage = 1000 
high_qty = tags[tags.usage>high_usage].shape[0]
print(f'High usage limit:{high_usage}, {high_qty} tags to delete')
remain_qty = tags.shape[0] - low_qty - high_qty
print(f'{remain_qty} tags remain')


Low usage limit:10, 28749 tags to delete
High usage limit:1000, 186 tags to delete
5317 tags remain


По результатам анализа у меня сложилось намерение отбросить те тэги, что описывают меньше десяти книг и больше 10000/10 = 1000 книг. 

In [69]:
tags_to_delete = set(list(tags[tags.usage<low_usage].tag_id) + list(tags[tags.usage>high_usage].tag_id))
print('Tags to detete:', len(tags_to_delete))

Tags to detete: 28935


In [73]:
# Сначала удаляем записи в справочнике tags
tags.drop(tags[tags.tag_id.isin(tags_to_delete)].index, axis=0, inplace=True) 
print('Tags after cleaning', tags.shape[0])
# теперь взаимодействия в справочнике book_tags
book_tags.drop( book_tags[book_tags.tag_id.isin(tags_to_delete)].index, axis=0, inplace=True)
print('Book Tag interactions after cleaning', book_tags.shape[0])

Tags after cleaning 5317
Book Tag interactions after cleaning 394096


In [117]:
tags.head()

Tags size 27


,tag_id,tag_name,usage
27,27,-fictitious,19
47,47,0,11
71,71,00-in-class,19
90,90,01-folklore,22
98,98,02-fantasy,48


In [120]:
len(tags.tag_id)

5317

Количество признаков существенно сократилось.  Теперь однако появилась проблема для использования нашего набора данных в некоторых пакетах - пространство tag_id оказалось разряженным - в нем есть пропуски значений, которые надо удалять для работы с большинством пакетов рекомендаций. 

Light_FM также требует непрерывных пространств признаков, однако предоставляет инструменты для их получения.  В этот пакет входят специальные конструкторы для построения матрицы взаимодействий и справочника признаков товарных единиц. 

Первым делом необходимо инициализировать класс Dataset, указав ему перечни пользователей, товарных единиц, свойств товарных единиц (и свойств пользователей - если есть).


In [121]:
from lightfm.data import Dataset
# Определяем экземпляр класса Dataset и указываем, что для него будут признаки свойств товарных единиц
dataset = Dataset(item_identity_features=True)
# Конструируем размерности и кодировщики для пользователей и идентификаторов книг

dataset.fit( ratings.user_id,   # пользователи 
             ratings.book_id,   # товарные единицы
             item_features= tags.tag_id)   # свойства товарных единиц - тэги
nu, ni = dataset.interactions_shape()
_, nf = dataset.item_features_shape()
print(f'Number of users {num_users}, number of items {num_items}, number of item features {nf} ')


Number of users 53424, number of items 10000, number of item features 13789 


Удачно получилось что исходная таблица ratings имеет три столбца, содержащие user id, item id и weight взаимодействия (оценку пользователя товарной единице).  Достаточно просто превратить это в итератор, полскольку этому конструктору надо в качестве аргумента подавать итератор или список из кортежа номера пользователя, номера товарной единицы и оценки пользователя товарной единице.  

In [122]:
interactions, weights = dataset.build_interactions(ratings.itertuples(index=False, name=None))
print(repr(interactions))

<53424x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 5976479 stored elements in COOrdinate format>


In [123]:
im = interactions.mean()
wm = weights.mean()
print(f'Mean of interactions {im:.4f}, mean of weights{wm:.4f}')

Mean of interactions 0.0112, mean of weights0.0439


Если задуматься, то размер созданной матрицы подразумевает наличие порядка 530 млн элементов. У нас всего 6 млн взаимодействий - значит и среднее взаимодействий должно быть порядка 1/100.  Цифры похожи на правду.  

Теперь разделим исходные данные на учебные и проверочные. Встроенная функция из пакета позволяет разделить сначала сами взаимодействия, затем их веса на два набора при фиксации одинакового random_state.

In [127]:
from lightfm.cross_validation import random_train_test_split

i_train, i_test = random_train_test_split(interactions, test_percentage=0.2, random_state=1234 )
w_train, w_test = random_train_test_split(weights, test_percentage=0.2, random_state=1234  )

In [132]:
print(repr(i_train))
print(repr(w_train))

<53424x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 4781183 stored elements in COOrdinate format>
<53424x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 4781183 stored elements in COOrdinate format>


In [133]:
i_train.col

array([5316, 9721,   32, ..., 1436, 6109,  548], dtype=int32)

In [134]:
w_train.col

array([6989, 5879, 3052, ..., 6092, 3793,  996], dtype=int32)

К сожалению, почему то матрицы interactions и weights не разделяются одинаковым образом и при совместнои их использовании для обучения модели возникает ошибка.  Пока не стал писать исправленную версию train_test_split и ограничился вариантом бинарной рекомендации. 

Теперь создаем матрицу, описывающую свойства товарных единиц (items). Здесь необходимо на вход итератор кортежей из двух значений - номер книги и список номеров признаков.  Сначала сделаем такой список, а потом подадим его на вход для построение матрицы свойств товарных единиц

In [126]:
item_features_list = [[i, list(book_tags[book_tags.book_id==i].tag_id)] for i in set(book_tags.book_id) ]
item_features = dataset.build_item_features(item_features_list ) 
print(repr(item_features))

<10000x13789 sparse matrix of type '<class 'numpy.float32'>'
	with 404080 stored elements in Compressed Sparse Row format>


In [136]:
from lightfm import LightFM

model = LightFM(loss='warp', #функция потерь для оптимизации precision@k
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели (50,100)

В модели LightFM есть гиперпараметр "размерность вектора представления даных".   Понятно что от него зависит точность и время обучения. Мне стало интересно просмотреть - как от длительности обучения растет точность. 

In [140]:
from lightfm.evaluation import precision_at_k
from time import perf_counter

In [142]:
hyper_param = [12,25,50,100]
for hp in hyper_param:
    model = LightFM(loss='warp', #функция потерь для оптимизации precision@k
                    random_state=13, #фиксируем случайное разбиение
                    learning_rate=0.05, #темп обучения
                    no_components=hp) #размерность вектора для представления данных в модели (50,100)
    t1 = perf_counter()
    model.fit( i_train, item_features= item_features, epochs=1, verbose = True)
    t2 = perf_counter()
    tt = t2-t1
    m = precision_at_k(model, i_test, item_features=item_features).mean()
    t3 = perf_counter()
    te = t3-t2
    print(f' {hp} components, precision @k = {m:.5f}, train time {tt:.2f} sec, evaluation time {te:.2f}')


Epoch: 100%|██████████| 1/1 [01:05<00:00, 65.75s/it]


 12 components, precision @k = 0.09113, train time 65.83 sec, evaluation time 311.32


Epoch: 100%|██████████| 1/1 [02:01<00:00, 121.39s/it]


 25 components, precision @k = 0.09507, train time 121.44 sec, evaluation time 635.26


Epoch: 100%|██████████| 1/1 [03:19<00:00, 199.15s/it]


 50 components, precision @k = 0.09854, train time 199.23 sec, evaluation time 530.15


Epoch: 100%|██████████| 1/1 [05:36<00:00, 336.14s/it]


 100 components, precision @k = 0.10147, train time 336.30 sec, evaluation time 493.53


На моем домашнем компьютере получилось достичь следующей точность precision@10: 
- 12 компонент - 0.0911 за 66 секунд,
- 25 компонент - 0.0951 за 121 секунду,
- 50 компонент - 0.0985 за 200 секунд,
- 100 компонент - 0.1015 за 336 секунд обучения модели.  

Качество получилось приемлемым - одна рекомендация из 10 предложенных за 5.5 мин обучения. 
Попробую долгий цикл обучения в 10 эпох при размерности компонента 100. 

In [144]:
model = LightFM(loss='warp', #функция потерь для оптимизации precision@k
                    random_state=13, #фиксируем случайное разбиение
                    learning_rate=0.05, #темп обучения
                    no_components=100) #размерность вектора для представления данных в модели (50,100)
t1 = perf_counter()
model.fit( i_train, item_features= item_features, epochs=10, verbose = True)
t2 = perf_counter()
tt = t2-t1
m = precision_at_k(model, i_test, item_features=item_features).mean()
t3 = perf_counter()
te = t3-t2
print(f' {hp} components, precision @k = {m:.5f}, train time {tt:.2f} sec, evaluation time {te:.2f}')

Epoch: 100%|██████████| 10/10 [47:06<00:00, 282.66s/it]


 100 components, precision @k = 0.11830, train time 2826.98 sec, evaluation time 515.16


После 55 минут ожидания модель показала точность precision@10 - 0.1183, что не намного лучше исходного варианта. 

In [146]:
print('Средний прирост качества за эпоху обучения:',(0.1183-0.1015)/9)

Средний прирост качества за эпоху обучения: 0.0018666666666666662


Можно предположить что за 80 часов обучения на моем компьютере точность рекомендаций превысит 0.2.  Невеселые перспективы. 

## Резюме  

Считается долго, качество на выходе не высокое. 